<a href="https://colab.research.google.com/github/JammieBurger/imlo-project-actual/blob/main/IMLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialising Training and Testing Data.

In [50]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
from torchvision.transforms import ToTensor

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25])])
training_data = datasets.Flowers102(
    root="data",
    split='train',
    transform=transform,
    download=True)
test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform=transform)

In [51]:
batch_size = 64

train_dataloader = DataLoader(training_data,batch_size)
test_dataloader = DataLoader(test_data,batch_size)
#for X, y in test_dataloader:
    #print(f"Shape of X [N, C, H, W]: {X.shape}")
    #print(f"Shape of y: {y.shape} {y.dtype}")
    #break
device = ("cuda"if torch.cuda.is_available()
                else "mps"
                if torch.backends.mps.is_available()
                else "cpu")
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(224*224*3, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 102),
            nn.ReLU(),
            nn.Linear(102,102),
            nn.ReLU(),
            nn.Linear(102,102)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Using cpu device


In [52]:
model = NeuralNetwork().to(device)
#print(model)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [53]:

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

Epoch 1
-------------------------------
loss: 4.616015  [   64/ 1020]
Test Error: 
 Accuracy: 0.7%, Avg loss: 4.954983 

Epoch 2
-------------------------------
loss: 5.332143  [   64/ 1020]
Test Error: 
 Accuracy: 0.7%, Avg loss: 4.713252 

Epoch 3
-------------------------------
loss: 5.114638  [   64/ 1020]
Test Error: 
 Accuracy: 0.6%, Avg loss: 4.663701 

Epoch 4
-------------------------------
loss: 4.671793  [   64/ 1020]
Test Error: 
 Accuracy: 0.9%, Avg loss: 4.635676 

Epoch 5
-------------------------------
loss: 4.396548  [   64/ 1020]
Test Error: 
 Accuracy: 0.8%, Avg loss: 4.620200 

Epoch 6
-------------------------------
loss: 4.222716  [   64/ 1020]
Test Error: 
 Accuracy: 1.4%, Avg loss: 4.639777 

Epoch 7
-------------------------------
loss: 4.136859  [   64/ 1020]
Test Error: 
 Accuracy: 1.6%, Avg loss: 4.674216 

Epoch 8
-------------------------------
loss: 4.013814  [   64/ 1020]
Test Error: 
 Accuracy: 2.0%, Avg loss: 4.695602 

Epoch 9
------------------------

KeyboardInterrupt: 